In [19]:
# Attention Mask : 실제토큰 1 / 0 패딩
# Token Type IDS(Segment IDs) : 두 개의 문장(A / B) 구성될때 각 토큰이 어느 문장에 속하는지 알려주는 임베딩
# CLS Token Pooling : [CLS] + token + [SEP]

In [20]:
# 1. Bert Tokenizer : 단어를 의미있는 조각(subword)로 나눕니다 unbelievable "un" 'believ" "able"
from transformers import BertTokenizer
# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentences = [
    "Hello world",
    "unbelievable performance",
    "COVID-19 pandamic"
]

# Bert Tokenizer 내부로직 과정
for sentence in sentences :
  # 토큰화
  tokens = tokenizer.tokenize(sentence)
  print(f'영문 : {sentence}')
  print(f'토큰 : {tokens}')

  # 변환
  ids = tokenizer.convert_tokens_to_ids(tokens)
  print(f'ID : {ids}')

  # 역변환
  decoded_string = tokenizer.decode(ids)
  print(f'역변환 : {decoded_string}')
  print()

  # 역변환을 했을때 대문자가아닌 소문자로 되있는 것을 확인 할 수 있다.

영문 : Hello world
토큰 : ['hello', 'world']
ID : [7592, 2088]
역변환 : hello world

영문 : unbelievable performance
토큰 : ['unbelievable', 'performance']
ID : [23653, 2836]
역변환 : unbelievable performance

영문 : COVID-19 pandamic
토큰 : ['co', '##vid', '-', '19', 'panda', '##mic']
ID : [2522, 17258, 1011, 2539, 25462, 7712]
역변환 : covid - 19 pandamic



In [21]:
# 2. Attention Mask : 실제단어 1, 패딩은 0 으로 표현
# 여기서 패딩에 사용되는 0은 아주 작은 값을 가지고 있다.
# -> softmax를 사용해서 0으로 변환된다^
sentences = [
    'short sentence',
    'This is a much longer sentence woth more words',
]

# 여러 문장을 한꺼번에 토크나이징하고 가장 긴 문장길이에 맞춰 자동 패딩 수행
encoded = tokenizer(
    sentences,
    padding = True,
    return_tensors='pt'
)
encoded

{'input_ids': tensor([[  101,  2460,  6251,   102,     0,     0,     0,     0,     0,     0,
             0,     0],
        [  101,  2023,  2003,  1037,  2172,  2936,  6251, 24185,  2705,  2062,
          2616,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [22]:
# token_type_ids : 두 문장을 입력할때 첫번째, 두번째 구분
from transformers import BertTokenizer
# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
sentence_A= "The weather is nice"
sentence_B= "Let's go for a walk"
# 두 문장을 하나의 입력으로 인코딩
encoded = tokenizer(
    sentence_A,
    sentence_B,
    padding = True,
    return_tensors="pt"
)
print(encoded["input_ids"][0])
tokens = tokenizer.convert_ids_to_tokens(encoded["input_ids"][0])
for token,token_id,type_id in zip(tokens, encoded["input_ids"][0], encoded['token_type_ids'][0]):
  segment ='문장 A' if type_id == 0 else '문장 B'
  if token == '[SEP]':
    segment = '구분자'
  elif token == '[CLS]':
    segment = '시작'
  print(f'{token:20s}{token_id.item():6d}({segment})')

  # 시작하는 토큰 하나와 끝나는 토큰 하나
  # 시작 문장1 끝, 시작할필요없이 바로 문장 2 끝 표시

tensor([ 101, 1996, 4633, 2003, 3835,  102, 2292, 1005, 1055, 2175, 2005, 1037,
        3328,  102])
[CLS]                  101(시작)
the                   1996(문장 A)
weather               4633(문장 A)
is                    2003(문장 A)
nice                  3835(문장 A)
[SEP]                  102(구분자)
let                   2292(문장 B)
'                     1005(문장 B)
s                     1055(문장 B)
go                    2175(문장 B)
for                   2005(문장 B)
a                     1037(문장 B)
walk                  3328(문장 B)
[SEP]                  102(구분자)


In [23]:
# [CLS] Token Pooling : BERT 첫번째 토큰 [CLS] 문서 전체의 요약 => 분류작업을 할때
# 이 토큰의 출력만 가져와서 분류기(classifier)에 연결
import torch
from transformers import BertTokenizer, BertModel
# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
sentence = 'BERT is amazing for NLP tasks'
# 인코딩
inputs = tokenizer(sentence,return_tensors='pt')
# BERT 통과
with torch.no_grad():
  outputs = model(**inputs)
# 출력 형태 확인
last_hidden_state = outputs.last_hidden_state
print(f'입력문장 : {sentence}')
print(f'last_hidden_state 형태 : {last_hidden_state.shape}')
print(f'batch_size = 1, sequence_length= {last_hidden_state.shape[1]} \
hidden_size = {last_hidden_state.shape[2]}')
# [CLS] 토큰 추출
cls_embedding = last_hidden_state[:, 0, :]
print(f'cls_embedding 형태 : {cls_embedding.shape}')
# 분류기 (2-class)
classifier = torch.nn.Linear(768,2)
logits = classifier(cls_embedding)
probs = torch.softmax(logits,dim=-1)
print(f'logits : {logits}')
print(f'probs : {probs}')
print(f'predicted class : {torch.argmax(probs).item()}')


입력문장 : BERT is amazing for NLP tasks
last_hidden_state 형태 : torch.Size([1, 9, 768])
batch_size = 1, sequence_length= 9 hidden_size = 768
cls_embedding 형태 : torch.Size([1, 768])
logits : tensor([[ 0.0855, -0.3234]], grad_fn=<AddmmBackward0>)
probs : tensor([[0.6008, 0.3992]], grad_fn=<SoftmaxBackward0>)
predicted class : 0


In [24]:
%pip install torch torchvision torchaudio

In [25]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
import torch

# 데이터
texts = [
    "This movie is faㅁntastic!",
    "Terrible film, waste of time.",
    "Amazing plot and great acting.",
    "Boring and predictable."
]
labels = [1, 0, 1, 0]  # 1=positive, 0=negative

# 토크나이저 모델
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 모델
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

# 데이터 셋
class SimpleDataset(Dataset):
  def __init__(self,texts,labels):
    self.encodings = tokenizer(texts,padding=True,truncation=True,return_tensors='pt')
    self.labels = labels
  def __getitem__(self, idx):
    item = {key: val[idx] for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

dataset = SimpleDataset(texts,labels)
loader = DataLoader(dataset, batch_size = 2)

# 학습설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = AdamW(model.parameters(),lr=1e-5)
# 모델을 미세조정(fine-tuning)합니다.
# 각 epoch마다 전체 데이터셋을 반복하며 배치 단위로:
# 1) 옵티마이저 초기화
# 2) 입력 데이터를 장치에 할당
# 3) 모델에 입력과 라벨 전달 후 손실 계산
# 4) 역전파로 가중치 업데이트
# 5) 배치 손실을 누적하여 epoch 평균 손실 계산
model.train()
for epoch in range(20):
  total_loss = 0
  for batch in loader:
    # 1) 옵티마이저 초기화
    optimizer.zero_grad()
    # 2) 입력 데이터를 장치에 할당
    inputs = {k:v.to(device) for k, v in batch.items() if k != 'labels'}
    labels = batch['labels'].to(device)
    # 3) 모델에 입력과 라벨 전달 후 손실 계산
    outputs = model(**inputs,labels=labels)
    loss = outputs.loss
    # 4) 역전파로 가중치 업데이트
    loss.backward()
    optimizer.step()
    # 5) 배치 손실을 누적하여 epoch 평균 손실 계산
    total_loss += loss.item()
    print(f'epoch :{epoch+1}, loss : {total_loss/len(loader)}')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


epoch :1, loss : 0.412182092666626
epoch :1, loss : 0.7363938391208649
epoch :2, loss : 0.3278186321258545
epoch :2, loss : 0.633906215429306
epoch :3, loss : 0.3037750720977783
epoch :3, loss : 0.5865463614463806
epoch :4, loss : 0.2668853998184204
epoch :4, loss : 0.5134022533893585
epoch :5, loss : 0.25739872455596924
epoch :5, loss : 0.47351598739624023
epoch :6, loss : 0.23052988946437836
epoch :6, loss : 0.47282877564430237
epoch :7, loss : 0.21135810017585754
epoch :7, loss : 0.4615785479545593
epoch :8, loss : 0.2339566946029663
epoch :8, loss : 0.4230566918849945
epoch :9, loss : 0.21104761958122253
epoch :9, loss : 0.3843788057565689
epoch :10, loss : 0.20207995176315308
epoch :10, loss : 0.42876556515693665
epoch :11, loss : 0.1410474181175232
epoch :11, loss : 0.32445839047431946
epoch :12, loss : 0.1274263858795166
epoch :12, loss : 0.31781457364559174
epoch :13, loss : 0.14033515751361847
epoch :13, loss : 0.288566455245018
epoch :14, loss : 0.16196230053901672
epoch :14,

In [30]:
# 추론
model.eval()
sample_sentence = ["The storyline was gripping and the acting was phenomenal!",
"I laughed, I cried, and I loved every moment of it.",
"Visually stunning, but the plot was kind of weak.",
"It felt like a waste of two hours, incredibly boring.",
"Terrible script and flat characters, I couldn't finish it."]

# labels = [1, 1, 0, 0, 0]
# 1=positive, 0=negative

# 토큰화
inputs = tokenizer(sample_sentence,
                   truncation = True,
                   padding = True,
                   return_tensors = 'pt')

# gpu / cpu 설정
inputs = {k:v.to(device) for k,v in inputs.items()}
# .items() :
# 딕셔너리(dictionary)의 키(key)와 값(value)을
# 동시에 가져오기 위해 사용하는 메서드

with torch.no_grad():
  outputs = model(**inputs)
  logits = outputs.logits
  probs = torch.softmax(logits,dim=-1)
  pred = torch.argmax(probs,dim=-1)
probs, pred # 1=positive, 0=negative
# 5개중 3개 맞춤

(tensor([[0.2205, 0.7795],
         [0.5105, 0.4895],
         [0.4587, 0.5413],
         [0.6915, 0.3085],
         [0.5720, 0.4280]], device='cuda:0'),
 tensor([1, 0, 1, 0, 0], device='cuda:0'))